In [1]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
# import asyncio
import re
import time
import os

from IPython.display import display, clear_output
from novelai_api.NovelAI_API import NovelAIAPI
from novelai_api.Preset import Model, Preset
from novelai_api.GlobalSettings import GlobalSettings
from novelai_api.Tokenizer import Tokenizer
from novelai_api.utils import b64_to_tokens
from novelai_api.BiasGroup import BiasGroup

import pandas as pd

In [6]:
# Settings

# Set method for authentication
# Posible options are:
# "enter_key" (enter your access key)
# "enter_login" (enter your username & pw)
# "enter_token" (enter your access token)
# "env_key" (read access key from environment variable NAI_KEY)
# "env_login" (read user from environment variable NAI_USERNAME and password from NAI_PASSWORD)
# "env_token"
auth_method = "env_token"
delay_time = 2
candidates_goal = 5
bias_strength = 0.1

In [9]:
auth = False
env = os.environ

# Init variable for login method
if auth_method == "enter_key":
    auth = input("Enter your NovelAI access key: ")
if auth_method == "enter_token":
    auth = input("Enter your NovelAI access token: ")
elif auth_method == "enter_login":
    auth = {}
    auth["user"] = input("Enter your NovelAI username: ")
    auth["pw"] = input("Enter your NovelAI password: ")
elif auth_method == "env_key":
    auth = env["NAI_KEY"]
elif auth_method == "env_token":
    auth = env["NAI_TOKEN"]
elif auth_method == "env_login":
    auth = {}
    auth["user"] = env["NAI_USERNAME"]
    auth["pw"] = env["NAI_PASSWORD"]
else:
    raise RuntimeError("Invalid value for 'auth_method'. Must be one of 'enter_key', 'enter_token', 'enter_login', env_key', 'env_token' or 'env_login")


In [10]:
async def nai_login (api, auth_method, auth):

    if auth_method == "enter_key" or auth_method == "env_key":
        await api.high_level.login_from_key(auth)
    elif auth_method == "enter_token" or auth_method == "env_token":
        await api.high_level.login_with_token(auth)
    elif auth_method == "enter_login" or auth_method == "env_login":
        await api.high_level.login(auth["user"], auth["pw"])

In [11]:
async def gen_attg_candidate(model=Model.Clio, prompt="[ Genre:", stop_sequences=[",", ";", " ]"], cut_stop_seq = True, auth_method="env_token", auth = None):
    # Initialize the NovelAI API
    api = NovelAIAPI()
    
    try:
        # Ensure you're logged in
        await nai_login(api, auth_method, auth)

        # Use the official "Edgewise" preset
        preset = Preset.from_official(model, "Edgewise")
        
        # Tokenize the stop sequences and set them for the preset
        stop_sequences_tokenized = [Tokenizer.encode(model, seq) for seq in stop_sequences]
        preset["stop_sequences"] = stop_sequences_tokenized
        
        # Create default global settings
        global_settings = GlobalSettings()

        gen = await api.high_level.generate(prompt, model, preset, global_settings)
        
        # After generating the text, remove the stop sequence
        generated_text = Tokenizer.decode(model, b64_to_tokens(gen["output"]))
        if cut_stop_seq:
            for seq in stop_sequences:
                generated_text = re.sub(re.escape(seq) + "$", "", generated_text).strip()
    
        return generated_text
    
    except Exception as e:
        raise Exception(f"Error generating text: {e}")

In [12]:
# Initialize an empty DataFrame
df = pd.DataFrame(columns=["phrase", "count"])

# Counter for total generations
total_generations = 0

# Loop until you have candidate_goal unique phrases
while len(df) < candidates_goal:
    total_generations += 1

    ## Clear the previous output
    clear_output(wait=True)
    
    print(f"Gen {total_generations}: Trying to gen phrase {len(df)+1}/{candidates_goal}...")

    try:
        phrase = await gen_attg_candidate(model=Model.Clio, prompt="[ Genre:", auth_method=auth_method, auth=auth)

        # Check if the phrase is already in the DataFrame
        if phrase in df["phrase"].values:
            df.loc[df["phrase"] == phrase, "count"] += 1
            print(f"Phrase '{phrase}' already exists. Incrementing count.")
        else:
            new_row = pd.DataFrame({"phrase": [phrase], "count": [1]})
            df = pd.concat([df, new_row], ignore_index=True)
            print(f"Added new phrase: '{phrase}'")

    except Exception as e:
        if "Anonymous quota reached" in str(e):
            print(f"Error: {e}")
            print("Anonymous rate limit reached. This indicates you are not properly authenticated. Check your authentication method. Aborting candidate search.")
            break
        else:
            print(f"Error: {e}")
            print("Aborting candidate search")
            break

    # Wait for delay_time seconds before the next generation attempt
    time.sleep(delay_time)

print("\nCandidate search complete!")
print(df)

Gen 9: Trying to gen phrase 5/5...
Added new phrase: 'science fiction'

Candidate search complete!
            phrase count
0           LitRPG     4
1           Action     1
2     Epic fantasy     1
3          Fantasy     2
4  science fiction     1
